# DATA MERGING AND VALIDATION FOR CREDIT RISK ANALYSIS

We will import and join the datasets and explore some missingness of the data

In [206]:
import pandas as pd
import pickle

In [207]:
trans = pd.read_csv('trans.asc',sep=';')
client = pd.read_csv('client.asc',sep=';')
account = pd.read_csv('account.asc',sep=';')
disp = pd.read_csv('disp.asc',sep=';')
order = pd.read_csv('order.asc',sep=';')
loan = pd.read_csv('loan.asc',sep=';')
card = pd.read_csv('card.asc',sep=';')
district = pd.read_csv('district.asc',sep=';')

### Reading and merging the loan and account -related datasets

In [208]:
df = pd.merge(loan, account,on='account_id', suffixes=['_loan','_acnt'], how='outer')
df = pd.merge(df, disp, on='account_id', how='outer')
df = pd.merge(df, client, on='client_id', how='outer', suffixes = ['_clnt','_acnt'])
df = pd.merge(df, district, left_on='district_id_clnt', right_on='A1', how='outer')
df = pd.merge(df, card, on='disp_id', how='outer', suffixes=['', '_card'])

### Read in and merge transaction data

In [209]:
trans = pd.read_csv('trans.asc', sep=';')

In [210]:
trans = pd.merge(trans, df, on='account_id', suffixes=['_trans', ''], how='left')
#Subset the data set to transactions for accounts with loans:
trans_loans = trans[~pd.isna(trans.loan_id)]

In [211]:
print('Total transactions: ', len(trans),'\nTransactions for accounts with a loan: ', len(trans_loans))

Total transactions:  1262625 
Transactions for accounts with a loan:  233627


### Explore proportional missingness in full data and the loan data

In [212]:
print(np.sum(trans.isna())/len(trans))

trans_id            0.000000
account_id          0.000000
date                0.000000
type_trans          0.000000
operation           0.172902
amount_trans        0.000000
balance             0.000000
k_symbol            0.451296
bank                0.733849
account             0.713619
loan_id             0.814967
date_loan           0.814967
amount              0.814967
duration            0.814967
payments            0.814967
status              0.814967
district_id_clnt    0.000000
frequency           0.000000
date_acnt           0.000000
disp_id             0.000000
client_id           0.000000
type                0.000000
birth_number        0.000000
district_id_acnt    0.000000
A1                  0.000000
A2                  0.000000
A3                  0.000000
A4                  0.000000
A5                  0.000000
A6                  0.000000
A7                  0.000000
A8                  0.000000
A9                  0.000000
A10                 0.000000
A11           

In [213]:
print(np.sum(trans_loans.isna())/len(trans_loans))

trans_id            0.000000
account_id          0.000000
date                0.000000
type_trans          0.000000
operation           0.163277
amount_trans        0.000000
balance             0.000000
k_symbol            0.482050
bank                0.732989
account             0.670102
loan_id             0.000000
date_loan           0.000000
amount              0.000000
duration            0.000000
payments            0.000000
status              0.000000
district_id_clnt    0.000000
frequency           0.000000
date_acnt           0.000000
disp_id             0.000000
client_id           0.000000
type                0.000000
birth_number        0.000000
district_id_acnt    0.000000
A1                  0.000000
A2                  0.000000
A3                  0.000000
A4                  0.000000
A5                  0.000000
A6                  0.000000
A7                  0.000000
A8                  0.000000
A9                  0.000000
A10                 0.000000
A11           

The missingness of credit card data is no issue, as the type and issue date of cards are likely not relevant to the problem. Bank and account columns report the bank and account of a partner in a transaction. As such, the specific bank and of a partner are likely not relevant and missingness should not be an issue.

Because we want to utilize the transactions for predicting credit defaults, transaction types are intuitively important. The operation and k_symbol columns describe the transaction type. We will explore the missing operation data further by examining, whethere there are cases where both operation and k_symbol are missing:

In [214]:
print('Percentage of cases where both operation and k_symbol are missing: ')
print(np.sum(trans.operation.isna() & trans.k_symbol.isna())/len(trans))
print('Percentage of cases where both operation and k_symbol are reported: ')
print(np.sum(~trans.operation.isna() & ~trans.k_symbol.isna())/len(trans))

print('Percentage of cases where both operation and k_symbol are missing: ')
print(np.sum(trans_loans.operation.isna() & trans_loans.k_symbol.isna())/len(trans))
print('Percentage of cases where both operation and k_symbol are reported: ')
print(np.sum(~trans_loans.operation.isna() & ~trans_loans.k_symbol.isna())/len(trans))

Percentage of cases where both operation and k_symbol are missing: 
0.0
Percentage of cases where both operation and k_symbol are reported: 
0.3758019998019998
Percentage of cases where both operation and k_symbol are missing: 
0.0
Percentage of cases where both operation and k_symbol are reported: 
0.06562597762597763


We see that, while operation and k_symbol are missing at times, there are no cases where neither one is present. Thus, missingness should be no issue.

Because missing data is not an issue and we do not need to use the transaction data to predict missing values, we can focus on the subset of the transactions data, where there are loans present.

### Datetime wrangling

Because we're interested in predicting bad loans, we should use transaction data from only prior to giving out the loan.
In order to work with the dates, we will first transform them to datetime format

In [215]:
trans_loans.columns

Index(['trans_id', 'account_id', 'date', 'type_trans', 'operation',
       'amount_trans', 'balance', 'k_symbol', 'bank', 'account', 'loan_id',
       'date_loan', 'amount', 'duration', 'payments', 'status',
       'district_id_clnt', 'frequency', 'date_acnt', 'disp_id', 'client_id',
       'type', 'birth_number', 'district_id_acnt', 'A1', 'A2', 'A3', 'A4',
       'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15',
       'A16', 'card_id', 'type_card', 'issued'],
      dtype='object')

In [216]:
trans_loans.date = pd.to_datetime(trans_loans.date, format='%y%m%d')
trans_loans.date_loan = pd.to_datetime(trans_loans.date_loan, format='%y%m%d')
trans_loans.issued = pd.to_datetime(trans_loans.issued.str[:6], format='%y%m%d')

In [217]:
trans_loans = trans_loans[trans_loans.date <= trans_loans.date_loan]

In [219]:
print('We end up with {} transactions for our final data.'.format(len(trans_loans)))

We end up with 66976 transactions for our final data.


### Dropping unnecessary columns

In [220]:
print('\n'.join(trans_loans.columns))

trans_id
account_id
date
type_trans
operation
amount_trans
balance
k_symbol
bank
account
loan_id
date_loan
amount
duration
payments
status
district_id_clnt
frequency
date_acnt
disp_id
client_id
type
birth_number
district_id_acnt
A1
A2
A3
A4
A5
A6
A7
A8
A9
A10
A11
A12
A13
A14
A15
A16
card_id
type_card
issued


## Export final transactions data



In [ ]:
with open('transactions_data','wb') as file:
    pickle.dump(trans_loans, file)

## Some additional data wrangling

,trans_id,account_id,date,amount_trans,balance,account,loan_id,date_loan,amount,duration,...,A6,A7,A8,A9,A10,A11,A13,A14,A16,card_id
count,2.336270e+05,233627.000000,233627.000000,233627.000000,233627.000000,7.707300e+04,233627.000000,233627.000000,233627.000000,233627.000000,...,233627.000000,233627.000000,233627.000000,233627.000000,233627.000000,233627.000000,233627.000000,233627.000000,233627.000000,47127.000000
mean,2.084692e+06,6034.333245,966293.534476,8239.434745,45520.418867,4.067135e+07,6216.416921,956830.527619,147130.014733,35.513121,...,21.533367,5.469137,1.682785,5.291794,68.555529,9551.048946,3.468643,122.373737,17582.671104,820.616738
std,1.115108e+06,3233.873384,13622.799197,11740.717669,24765.050048,3.272112e+07,672.007484,14668.160340,111462.998365,17.255362,...,16.112486,4.601195,1.071525,2.907060,20.480304,1356.102185,2.142688,23.791757,32637.127783,327.342498
min,2.760000e+02,2.000000,930113.000000,0.000000,-19310.000000,0.000000e+00,4959.000000,930705.000000,4980.000000,12.000000,...,0.000000,0.000000,0.000000,1.000000,33.900000,8110.000000,0.430000,81.000000,888.000000,16.000000
25%,1.113868e+06,3115.000000,960307.000000,199.500000,27441.450000,7.603472e+06,5611.000000,941003.000000,67464.000000,24.000000,...,8.000000,2.000000,1.000000,4.000000,52.700000,8546.000000,1.960000,106.000000,2252.000000,587.000000
50%,2.279742e+06,6148.000000,970508.000000,3600.000000,40934.000000,3.866864e+07,6253.000000,960119.000000,108144.000000,36.000000,...,23.000000,5.000000,1.000000,6.000000,61.900000,8994.000000,3.490000,117.000000,3868.000000,890.000000
75%,3.091876e+06,8784.000000,980305.000000,10530.000000,59333.200000,6.941577e+07,6766.000000,970518.000000,203940.000000,48.000000,...,33.000000,8.000000,2.000000,7.000000,87.700000,9897.000000,4.720000,137.000000,6872.000000,1081.000000
max,3.682987e+06,11362.000000,981231.000000,87300.000000,209637.000000,9.999420e+07,7308.000000,981208.000000,590820.000000,60.000000,...,70.000000,20.000000,5.000000,11.000000,100.000000,12541.000000,9.400000,167.000000,99107.000000,1247.000000
